# Day 15: Chiton
This has to be a problem for 
[Dijkstra's Algorithm](https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm)
([Computerphile's breakdown](https://www.youtube.com/watch?v=GazC3A4OQTE))
but Dijkstra doesn't account for weights per point it just does shortes path.

A* does account for weights
([Computerphile's breakdown](https://www.youtube.com/watch?v=ySN5Wnu88nE))

In [ ]:
let int (ch:char) :int = int ch - int '0'
let array = Seq.map int
let sample = "1163751742
1381373672
2136511328
3694931569
7463417111
1319128137
1359912421
3125421639
1293138521
2311944581"
let cleanData :int[,] = sample.Split(Environment.NewLine) |> Array.map array |> array2D
let finish :int*int = (cleanData.GetUpperBound(0), cleanData.GetUpperBound(1))
let thrd (_,_,t,_) = t
let fth (_,_,_,t) = t

let sub (x1,y1) (x2,y2) = (x1 - x2, y1 - y2)
let square (x,y) = (pown x 2, pown y 2)
/// A truncated version of pythag as we just want a distance modifier
let dist a b = (a,b) ||> sub ||> (+)
let getNeighbors x y = 
    [|
        for a = max (x-1) 0 to min (x+1) (fst finish) do
            for b = max (y-1) 0 to min (y+1) (snd finish) do
                if (a = x && b <> y) || (a <> x && b = y) then //  Don't consider diagonals for now
                // if a <> x || b <> y then // Take all neighbors excluding the given point
                    (a,b, cleanData.[a,b], (dist finish (a,b)) + cleanData.[a,b])

    |]


let astar =
    let mutable depth = 0
    let rec f path arr =
        if depth < 100 then
            let h = Array.find (fun a -> Array.contains a path |> not) arr
            // path |> Array.sortBy fth |> display |> ignore
            // h |> display |> ignore
            let t = Array.tail arr
            let (x,y,_,_) = h
            if x = fst finish && y = snd finish then path
            else
                depth <- depth + 1
                getNeighbors x y |> Array.append t |> Array.sortBy fth
                |> f [|yield! path; yield h|]
        else path

    getNeighbors 0 0 |> f [|(0,0,cleanData.[0,0],(fst finish + snd finish) + cleanData.[0,0])|]

astar

index,Item1,Item2,Item3,Item4
0,0,0,1,19
1,0,1,1,18
2,1,0,1,18
3,2,0,2,18
4,2,1,1,16
5,2,2,3,17
6,3,0,3,18
7,1,1,3,19
8,2,3,6,19
9,1,3,1,15
